# Questions
- which TFs make into a regulon and which do not? 

- TFBS database
- level of expression 
- additional criteria? 

In [ ]:
import os
import numpy as np
import pandas as pd
import scanpy as sc
import anndata 
import seaborn as sns
from scipy.stats import zscore
import matplotlib.pyplot as plt
import collections
from natsort import natsorted

from scipy import stats
from scipy import sparse
from sklearn.decomposition import PCA
from umap import UMAP
from statsmodels.stats.multitest import multipletests

from matplotlib.colors import LinearSegmentedColormap

from scroutines.config_plots import *
from scroutines import powerplots # .config_plots import *
from scroutines import pnmf
from scroutines import basicu
from scroutines.gene_modules import GeneModules  

import atac_utils

In [ ]:
ddir = '/u/home/f/f7xiesnm/v1_multiome/juyoun/' 
f = ddir+'L23alltime_eRegulon_metadata_unfiltered.csv'
# scenic metadata
df_scenic = pd.read_csv(f, index_col=0)
assert np.all(df_scenic['Region_signature_name'].apply(lambda x: x.split('(')[0][:-1]).values
           == df_scenic['Gene_signature_name'].apply(lambda x: x.split('(')[0][:-1]).values)
df_scenic['Consensus_name'] = df_scenic['Gene_signature_name'].apply(lambda x: x.split('(')[0][:-1])
df_scenic['signs'] = df_scenic['Consensus_name'].apply(lambda x: x[-3:])
df_scenic

In [ ]:
df_reg = df_scenic.groupby(['TF', 'signs', 'is_extended',  'Consensus_name',]).first()[['Region_signature_name', 'Gene_signature_name', ]].sort_values('TF')

scenic_regions = np.sort(df_scenic['Region'].unique()) # .shape
scenic_genes = np.sort(df_scenic['Gene'].unique()) # .shape
scenic_tfs = np.sort(df_scenic['TF'].unique())

num_reg    = len(df_reg)
num_tf     = len(scenic_tfs)
num_gene = len(scenic_genes)
num_region = len(scenic_regions)
print(num_reg, num_tf, num_gene, num_region)
df_reg

In [ ]:
df_reg.reset_index().groupby('TF').size().sort_values()

In [ ]:
df_reg.loc['Meis2']

# trim
- a regulon can be uniquely identified by (TF, signs, is_extended) 
- if two regulons share the same (TF, signs), only keep the ones that is 'is_extended' 

In [ ]:
df_reg_out = df_reg.reset_index().sort_values(['TF', 'signs', 'is_extended']).reset_index().drop('index', axis=1)
df_reg_out

In [ ]:
df_reg2 = df_reg_out.sort_values(['TF', 'signs', 'is_extended'], ascending=False).reset_index().groupby(['TF', 'signs']).first() #groupby('TF').size()
df_reg2

In [ ]:
df_reg2.loc['Meis2'] # .groupby(['TF', 'signs']).size().sort_values()

In [ ]:
trimmed_regulons = df_reg2['Consensus_name']
df_scenic2 = df_scenic[df_scenic['Consensus_name'].isin(trimmed_regulons)]
df_scenic2

In [ ]:
df_scenic2['Consensus_name'].unique().shape

In [ ]:
df_reg_out.shape, df_reg2.shape

# df_reg_new
- remove the name "extended"

In [ ]:
df_reg_new = df_reg2.copy()
df_reg_new['Consensus_name'] = df_reg_new['Consensus_name'].apply(lambda x: "".join(x.split('_extended')))
df_reg_new['Region_signature_name'] = df_reg_new['Region_signature_name'].apply(lambda x: "".join(x.split('_extended')))
df_reg_new['Gene_signature_name'] = df_reg_new['Gene_signature_name'].apply(lambda x: "".join(x.split('_extended')))
df_reg_new

In [ ]:
df_scenic_new = df_scenic2.copy()
df_scenic_new['Consensus_name'] = df_scenic_new['Consensus_name'].apply(lambda x: "".join(x.split('_extended')))
df_scenic_new['Region_signature_name'] = df_scenic_new['Region_signature_name'].apply(lambda x: "".join(x.split('_extended')))
df_scenic_new['Gene_signature_name'] = df_scenic_new['Gene_signature_name'].apply(lambda x: "".join(x.split('_extended')))
df_scenic_new

In [ ]:
df_reg_out.shape, df_reg2.shape, df_scenic2.shape, df_reg_new.shape, df_scenic_new.shape

In [ ]:
df_reg_out   .to_csv('/u/home/f/f7xiesnm/v1_multiome/juyoun/regulons_l23alltime_all.csv')
df_reg2      .to_csv('/u/home/f/f7xiesnm/v1_multiome/juyoun/regulons_l23alltime_trimmed.csv')
df_scenic2   .to_csv('/u/home/f/f7xiesnm/v1_multiome/juyoun/regulons_l23alltime_trimmed_bigtable.csv')
df_reg_new   .to_csv('/u/home/f/f7xiesnm/v1_multiome/juyoun/regulons_l23alltime_trimmed_cleaned.csv')
df_scenic_new.to_csv('/u/home/f/f7xiesnm/v1_multiome/juyoun/regulons_l23alltime_trimmed_cleaned_bigtable.csv')

# get region_list

In [ ]:
scenic_regions = np.sort(df_scenic_new['Region'].unique()) # .shape
scenic_regions.shape

In [ ]:
outbed = '/u/home/f/f7xiesnm/v1_multiome/juyoun/scenic_regions_v2.bed'
df_scenic_regions = pd.DataFrame() 
df_scenic_regions['chr']   = [i.split(':')[0] for i in scenic_regions]
df_scenic_regions['start'] = [i.split(':')[1].split('-')[0] for i in scenic_regions]
df_scenic_regions['end']   = [i.split(':')[1].split('-')[1] for i in scenic_regions]
df_scenic_regions.to_csv(outbed, sep='\t', index=False, header=False, )
df_scenic_regions

In [ ]:
!wc -l $outbed
!head $outbed